### Merging all categories grouped on HADM_ID

In [ ]:
import pandas as pd
import numpy as np
from tqdm.autonotebook import tqdm
import re
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# create output path
mypath_input = "/home/jupyter/datasets/training_data/data_before_24hrs_icu/data_grouped_HADM_ID/"
mypath_output = "/home/jupyter/datasets/training_data/data_before_24hrs_icu/data_grouped_HADM_ID/"
os.makedirs(mypath_output, exist_ok=True)

In [ ]:
file_list = list(filter(lambda k: '.json' in k, os.listdir(mypath_input)))

In [ ]:
file_list

In [ ]:
#Remove chartevents and all_events fromt he files list
if 'chartevents.json' in file_list:
    file_list.remove('chartevents.json')
    print('Chartevents removed from list')
else :
    print('Chartevents not found in list')
if 'all_events.json' in file_list:
    file_list.remove('all_events.json')
    print('all_events removed from list')
else :
    print('all_events not found in list')
    

In [ ]:
df = pd.read_json(mypath_input+"chartevents.json")

In [ ]:
file_list

In [ ]:
for i in tqdm(file_list):
    x = re.sub('.json','',i)
    df1 = pd.read_json(mypath_input+i)
    df1 = df1.rename({'events':x}, axis = 'columns')
    df = df.merge(df1[['HADM_ID',x]], how='left',left_on = 'HADM_ID',right_on = 'HADM_ID')

In [ ]:
column_list = df.columns.tolist()

In [ ]:
column_list.remove('HADM_ID')
column_list.remove('HOSPITAL_EXPIRE_FLAG')
column_list.remove('SUBJECT_ID')
column_list

In [ ]:
df.head()

In [ ]:
df.fillna('NaN', inplace=True)

In [ ]:
for i in tqdm(column_list):
    df[i] = df[i].fillna(0).map(lambda x: [x] if x == 'NaN' else x)

In [ ]:
df.head()

In [ ]:
df.to_json(mypath_output+"all_events.json")